In [ ]:
!mkdir output
!rm -Rf output/* # 出力結果リセット

In [22]:
#複製元の画像取得
import io
import urllib.request
from PIL import Image
f = io.BytesIO(urllib.request.urlopen("http://books.google.com/books/content?id=anCgswEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api").read())
img = Image.open(f)
img.save("input/production_micro_ready.jpg")

f = io.BytesIO(urllib.request.urlopen("http://books.google.com/books/content?id=c6dMswEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api").read())
img = Image.open(f)
img.save("input/dev_ops.jpg")

f = io.BytesIO(urllib.request.urlopen("http://books.google.com/books/content?id=l5VRDwAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api").read())
img = Image.open(f)
img.save("input/server_less.jpg")

In [23]:
# 画像複製のコード
import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

def draw_images(generator, x, dir_name, index):
    # 出力ファイルの設定
    save_name = 'ext-'
    if not(os.path.exists('output/' + str(index))):
        os.mkdir('output/' + str(index))

    g = generator.flow(x, batch_size=1, save_to_dir='output/' + str(index), save_prefix=save_name, save_format='jpg')

    # 1つの入力画像から何枚複製するかを指定
    # g.next()の回数分拡張される
    for i in range(50):
        bach = g.next()
        
def main():
    # 複製する画像群の読み込み
    images = glob.glob(os.path.join('./input/', "*"))

    # 複製する際の設定
    generator = ImageDataGenerator(
                    rotation_range=45, # 45°まで回転
                    width_shift_range=0.3, # 水平方向にランダムでシフト
                    height_shift_range=0.3, # 垂直方向にランダムでシフト
                    channel_shift_range=5.0, # 色調をランダム変更
                    shear_range=0.1, # 斜め方向に引っ張る
                    horizontal_flip=False, # 垂直方向にランダムで反転
                    vertical_flip=False # 水平方向にランダムで反転
                    )

    # 読み込んだ画像を順に複製
    for i in range(len(images)):
        img = load_img(images[i])
        # 画像を配列化して転置
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # 画像の複製
        draw_images(generator, x, "", i)

In [29]:
# 画像複製の実行
main()

In [ ]:
# 出力結果の確認
!ls -lR input output

In [31]:
# 複製した画像のndarray化
X = []
Y = []

images = glob.glob(os.path.join('./output/0/', "*.jpg"))
for img in images:
  l_img = load_img(img, target_size=[299,299])
  X.append(img_to_array(l_img))
  Y.append(0)
  
images = glob.glob(os.path.join('./output/1/', "*.jpg"))
for img in images:
  l_img = load_img(img, target_size=[299,299])
  X.append(img_to_array(l_img))
  Y.append(1)
  
images = glob.glob(os.path.join('./output/2/', "*.jpg"))
for img in images:
  l_img = load_img(img, target_size=[299,299])
  X.append(img_to_array(l_img))
  Y.append(2)
  
X = np.asarray(X)
Y = np.asarray(Y)

# npz形式で保存
np.savez('datasets2.npz',features = X, labels = Y)

In [ ]:
# GCSのバケットに保存
!gsutil cp datasets2.npz gs://asobiba-ml-bucket/datasets.npz